## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [92]:
%%sql
postgresql:///animal_shelters

CREATE TABLE main AS
SELECT *, DATE_PART('year',AGE('2021-12-31',birthdate::DATE))::int AS age
FROM animals;

(psycopg2.errors.DuplicateTable) relation "main" already exists

[SQL: CREATE TABLE main AS
SELECT *, DATE_PART('year',AGE('2021-12-31',birthdate::DATE))::int AS age
FROM animals;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [93]:
%%sql
postgresql:///animal_shelters

CREATE TABLE main_sizes AS
SELECT m.animaltype, lc.costs AS location_cost, ac.costs AS age_cost,
    CASE 
    WHEN animaltype = 'Dog' AND weight <= 10 THEN 'Small'
    WHEN animaltype = 'Dog' AND weight > 10 AND weight <= 30 THEN 'Medium'
    WHEN animaltype = 'Dog' AND weight > 30 THEN 'Large'
    WHEN animaltype = 'Cat' AND weight <= 5 THEN 'Small'
    WHEN animaltype = 'Cat' AND weight > 5 AND weight <= 7 THEN 'Medium'
    WHEN animaltype = 'Cat' AND weight > 7 THEN 'Large'
    WHEN animaltype = 'Bird' AND weight <= 0.7 THEN 'Small'
    WHEN animaltype = 'Bird' AND weight > 0.7 AND weight <= 1.1 THEN 'Medium'
    WHEN animaltype = 'Bird' AND weight > 1.1 THEN 'Large'
    ELSE NULL END as size
FROM main AS m
INNER JOIN location_costs AS lc ON m.location = lc.location
INNER JOIN age_costs as ac ON m.age = (ac.age::int)
WHERE animalid NOT IN (SELECT sponsorid FROM sponsored_animals);

(psycopg2.errors.DuplicateTable) relation "main_sizes" already exists

[SQL: CREATE TABLE main_sizes AS
SELECT m.animaltype, lc.costs AS location_cost, ac.costs AS age_cost,
    CASE 
    WHEN animaltype = 'Dog' AND weight <= 10 THEN 'Small'
    WHEN animaltype = 'Dog' AND weight > 10 AND weight <= 30 THEN 'Medium'
    WHEN animaltype = 'Dog' AND weight > 30 THEN 'Large'
    WHEN animaltype = 'Cat' AND weight <= 5 THEN 'Small'
    WHEN animaltype = 'Cat' AND weight > 5 AND weight <= 7 THEN 'Medium'
    WHEN animaltype = 'Cat' AND weight > 7 THEN 'Large'
    WHEN animaltype = 'Bird' AND weight <= 0.7 THEN 'Small'
    WHEN animaltype = 'Bird' AND weight > 0.7 AND weight <= 1.1 THEN 'Medium'
    WHEN animaltype = 'Bird' AND weight > 1.1 THEN 'Large'
    ELSE NULL END as size
FROM main AS m
INNER JOIN location_costs AS lc ON m.location = lc.location
INNER JOIN age_costs as ac ON m.age = (ac.age::int)
WHERE animalid NOT IN (SELECT sponsorid FROM sponsored_animals);]
(Background on this erro

In [94]:
%%sql
postgresql:///animal_shelters

SELECT column_name, data_type FROM information_schema.columns WHERE 
table_name = 'main_costs';

5 rows affected.


column_name,data_type
animaltype,character varying
location_cost,integer
age_cost,integer
size,text
costs,integer


In [95]:
%%sql
postgresql:///animal_shelters

CREATE TABLE main_costs AS
SELECT ms.animaltype, ms.location_cost, ms.age_cost, ms.size, sc.costs
FROM main_sizes AS ms
JOIN size_costs AS sc ON ms.size = sc.size
WHERE ms.animaltype = sc.animaltype;

(psycopg2.errors.DuplicateTable) relation "main_costs" already exists

[SQL: CREATE TABLE main_costs AS
SELECT ms.animaltype, ms.location_cost, ms.age_cost, ms.size, sc.costs
FROM main_sizes AS ms
JOIN size_costs AS sc ON ms.size = sc.size
WHERE ms.animaltype = sc.animaltype;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [96]:
%%sql
postgresql:///animal_shelters

CREATE TABLE main_total AS
SELECT LOWER(animaltype) as animaltype, LOWER(size) as size, 
    SUM(location_cost + age_cost + costs) as total
FROM main_costs
GROUP BY animaltype, size
ORDER BY animaltype, size DESC;

(psycopg2.errors.DuplicateTable) relation "main_total" already exists

[SQL: CREATE TABLE main_total AS
SELECT LOWER(animaltype) as animaltype, LOWER(size) as size, 
    SUM(location_cost + age_cost + costs) as total
FROM main_costs
GROUP BY animaltype, size
ORDER BY animaltype, size DESC;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [97]:
%%sql
postgresql:///animal_shelters
    
SELECT animaltype,size,total, ROUND(total / SUM(total) OVER (),2) as percentage
FROM main_total;

9 rows affected.


animaltype,size,total,percentage
bird,small,1615,0.00
bird,medium,3460,0.00
bird,large,7770,0.00
cat,small,518015,0.15
cat,medium,250575,0.07
cat,large,439490,0.13
dog,small,336530,0.10
dog,medium,941895,0.27
dog,large,977665,0.28
